In [2]:
import pandas as pd
import numpy as np
import pulp

In [4]:
finished_raw_data = pd.read_excel('finished_raw_ratio_rate.xlsx', engine = 'openpyxl')
sales_data = pd.read_excel('Average_sales_product.xlsx', engine = 'openpyxl')
raw_rate = pd.read_excel('raw_rate.xlsx', engine = 'openpyxl')

In [5]:
print(finished_raw_data.columns)
print(sales_data.columns)
print(raw_rate.columns)

Index(['Unnamed: 0', 'goods', 'raw', 'xqty/unit', 'xrate'], dtype='object')
Index(['Unnamed: 0', 'xitem', 'xdesc', 'xqty', 'xval', 'xlineamt'], dtype='object')
Index(['Unnamed: 0', 'raw', 'xrate'], dtype='object')


In [6]:
# define data
products = sales_data['xitem'].unique()
demand = dict(zip(sales_data['xitem'], sales_data['xqty']))

# load raw materials relations 
raw_materials = finished_raw_data['raw'].unique()

In [7]:
material_requirements = finished_raw_data.groupby('goods').apply(
    lambda x: dict(zip(x['raw'], x['xqty/unit']))
).to_dict()

In [8]:
model = pulp.LpProblem("Raw_Material_Procurement", pulp.LpMinimize)
raw_material_vars = {material: pulp.LpVariable(f"quantity_{material}", lowBound=0) for material in raw_materials}

# objective minimize total raw materials purchased 
model += pulp.lpSum(raw_material_vars[material] for material in raw_materials), "Total_Raw_Material"


for material in raw_materials:
    model += pulp.lpSum(
        material_requirements[product].get(material, 0.0) * demand[product] 
        for product in products if product in material_requirements
    ) == raw_material_vars[material], f"Material_{material}_constraint"


model.solve()


print("Optimal quantities of raw materials to purchase:")
for material in raw_materials:
    print(f"{material}: {raw_material_vars[material].varValue}")

Optimal quantities of raw materials to purchase:
RZ000193: 528.67
RZ000002: 101.1727
RZ000004: 10.224278
RZ000309: 1.6715518
RZ000230: 29.313899
RZ000235: 687.39244
RZ000177: 299.67
RZ000228: 8.0278092
RZ000176: 299.67
RZ000304: 1256.7591
RZ000410: 4666.3799
RZ000030: 11134.043
RZ000282: 225.8697
RZ000273: 244.1401
RZ000199: 760.33465
RZ000006: 69.113557
RZ000157: 5139.0
RZ000001: 37.755082
RZ000411: 3959.2202
RZ000315: 5547.35
RZ000178: 109.0
RZ000160: 4.8316015
RZ000005: 4.8374227
RZ000007: 35.152076
RZ000181: 2002.0
RZ000180: 2002.0
RZ000159: 3.2560464
RZ000158: 5.0647193
RZ000003: 41.30668
RZ000303: 1363.2069
RZ000365: 5.33
RZ000355: 56.33
RZ000275: 36.0
RZ000274: 36.0
RZ000277: 1243.67
RZ000276: 1243.67
RZ000310: 5.2305113
RZ000270: 0.18039437
RZ000279: 111.0
RZ000278: 111.0
RZ000009: 4.384399
RZ000319: 1969.34
RZ000311: 4.3245305
RZ000470: 1892.67
RZ000453: 25.510243
RZ000254: 2.1677656
RZ000012: 4.6023437
RZ000320: 1969.34
RZ000413: 122.33
RZ000326: 122.33
RZ000454: 3.4502676
RZ

In [9]:
results = {material: raw_material_vars[material].varValue for material in raw_materials}
results_df = pd.DataFrame(list(results.items()), columns=['Raw Material', 'Optimal Quantity'])

In [19]:
results_df

,Raw Material,Optimal Quantity
0,RZ000193,528.670000
1,RZ000002,101.172700
2,RZ000004,10.224278
3,RZ000309,1.671552
4,RZ000230,29.313899
...,...,...
116,RZ000479,14.670000
117,RZ000481,0.124855
118,RZ000476,83.670000
119,RZ000478,83.670000


In [14]:
test = finished_raw_data[finished_raw_data['raw']=='RZ000193']

In [16]:
sales_data[sales_data['xitem'].isin(test['goods'])]['xqty'].sum()

528.6700000000001